IMPORTACIÖN DE LIBS Y FORMATEO DEL DATASET

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import special
import math
import csv
import disdros

plt.style.use('classic')
path_data = 'C:/Users/pc/EAFITDATA/Disdrometros/Solicitud_Disdros'
path_results = 'C:/Users/pc/EAFITDATA/Disdrometros/Results'
dsd_file = '77_2013-05-01.csv'
dsd_number = (dsd_file[0:dsd_file.find("_")])
print(dsd_number)
savefigs = '../figs/{}'.format(dsd_number)
savepks = '../pks/{}'.format(dsd_number)

77


CREACIÓN DEL PRIMER DF DE LECTURA SETEO DE VARIABLES DE VELOCIDAD Y TAMAÑO DE GOTAS DEFINIDO EN EL MANUAL DE DISDROS

In [2]:
num_gotas = 1
print (savefigs, savepks)

chunks = pd.read_csv('{}/{}'.format(path_data, dsd_file), delimiter=',', header=None, chunksize=10 ** 4,
                     low_memory=False, na_values='None')

dsd_final = pd.DataFrame()
dsd_results = pd.DataFrame()

../figs/77 ../pks/77


In [4]:
i = 0
for chunk in chunks:# UN CHUNK ES DIVIDIR EL CSV EN PARTES PARA QUE NO ROMPA LA RAM POR ESO SE TIENE QUE PROCESAR VARIAS VECES
    i += 1
    #PREPROCESAMIENTO
    dsd_data_final= disdros.preprocess_csv(chunk)
    #HEADER MULTIPLE PARA TENER LAS LONGITUDES Y VELOCIDADES
    idx_header = pd.MultiIndex.from_product([sorted(disdros.list_size), sorted(disdros.list_speed)], names=['Size', 'Speed'])
    dsd_data_final.columns = idx_header
    # SE OBTIENE LA SUMA DE EVENTOS O GOTAS Y SE COMPARA CON LAS GOTAS DESEADAS EN ESTE CASO QUE SEAN MAS DE 1 
    # SE EXTRAEN SOLO LOS INDICES CON DATOS EN ESA SUMA PARA SACARLES LAS VARIABLES FISICAS
    idx_events = disdros.getIndexOfCondition(dsd_data_final,num_gotas)
    delta_diam = disdros.delta_diam
    diameters = disdros.diameters
    for event in idx_events:
        sr_nd = pd.Series(index=sorted(disdros.list_size))
        sr_nt = pd.Series(index=sorted(disdros.list_size))
        sr_w = pd.Series(index=sorted(disdros.list_size))
        sr_R = pd.Series(index=sorted(disdros.list_size))
        sr_Z = pd.Series(index=sorted(disdros.list_size))
        for size in disdros.list_size:
            df_size = dsd_data_final.xs(size, level=0, axis=1)
            sr_event = df_size.loc[event]
            dsize = disdros.list_size[size]

            nd = disdros.calc_nd(sr_event, disdros.list_speed, dsize)
            sr_nd.loc[size] = nd

            nt = disdros.calc_nt(sr_event, disdros.list_speed, dsize)
            sr_nt.loc[size] = nt

            W = disdros.calc_W(sr_event, disdros.list_speed, dsize)
            sr_w.loc[size] = W

            R = disdros.calc_R(sr_event, disdros.list_speed, dsize)
            sr_R.loc[size] = R

            Z = disdros.calc_R(sr_event, disdros.list_speed, dsize)
            sr_Z.loc[size] = Z
        try:
            # PARAMETROS M
            M0= disdros.calc_MParams(sr_nd,0)
            dsd_results.loc[event, 'M0'] = M0
            M1= disdros.calc_MParams(sr_nd,1)
            dsd_results.loc[event, 'M1'] = M1
            M2= disdros.calc_MParams(sr_nd,2)
            dsd_results.loc[event, 'M2'] = M2
            M3= disdros.calc_MParams(sr_nd,3)
            dsd_results.loc[event, 'M3'] = M3
            M4= disdros.calc_MParams(sr_nd,4)
            dsd_results.loc[event, 'M4'] = M4
            M5= disdros.calc_MParams(sr_nd,5)
            dsd_results.loc[event, 'M5'] = M5
            M6= disdros.calc_MParams(sr_nd,5)
            dsd_results.loc[event, 'M6'] = M6
            # MICROFISICA DE PRECIPITACION
            NT_dsd = sr_nt.sum()
            NT_dsd_1 = M0
            W_dsd = (math.pi / 6.0) * 10 ** (-3) * sr_w.sum()
            R_dsd = 6*math.pi * 10 ** (-4.0) * sr_R.sum()
            Z_dsd = 10*np.log10(sr_Z.sum())
            Z_dsd_cal = 10*np.log10(M6)
            Da_dsd = M1 / M0
            De_dsd = M3 / M2
            Dm_dsd = M4 / M3

            dsd_results.loc[event, 'NT_dsd'] = NT_dsd
            dsd_results.loc[event, 'NT_dsd_1'] = NT_dsd_1
            dsd_results.loc[event, 'W_dsd'] = W_dsd
            dsd_results.loc[event, 'Z_dsd'] = Z_dsd
            dsd_results.loc[event, 'Z_dsd_cal'] = Z_dsd_cal
            dsd_results.loc[event, 'R_dsd'] = R_dsd
            dsd_results.loc[event, 'Da_dsd'] = Da_dsd
            dsd_results.loc[event, 'De_dsd'] = De_dsd
            dsd_results.loc[event, 'Dm_dsd'] = Dm_dsd
            dsd_results.append(dsd_results)
        except (ZeroDivisionError,OverflowError) as e:
            print (e)
            continue



C:\Users\pc\AppData\Local\Temp\ipykernel_18676\1528060341.py:15: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  sr_nd = pd.Series(index=sorted(disdros.list_size))
C:\Users\pc\AppData\Local\Temp\ipykernel_18676\1528060341.py:16: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  sr_nt = pd.Series(index=sorted(disdros.list_size))
C:\Users\pc\AppData\Local\Temp\ipykernel_18676\1528060341.py:17: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  sr_w = pd.Series(index=sorted(disdros.list_size))
C:\Users\pc\AppData\Local\Temp\ipykernel_18676\1528060341.py:18: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future

In [8]:
dsd_results.sort_index(inplace=True)
dsd_results.dropna(how='all', inplace=True)
xls_data = pd.ExcelWriter('{0}/dsd_results_{2}_{3}.xlsx'.format(path_results, dsd_number, dsd_number, num_gotas))
dsd_results.to_excel(xls_data)
xls_data.save()

C:\Users\pc\AppData\Local\Temp\ipykernel_18676\3292288580.py:5: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  xls_data.save()


Se generan los dos headers multiples de datos en bruto según manual de disdrometros

In [9]:
dsd_results

,M0,M1,M2,M3,M4,M5,M6,NT_dsd,NT_dsd_1,W_dsd,Z_dsd,Z_dsd_cal,R_dsd,Da_dsd,De_dsd,Dm_dsd
2013-05-01 00:20:04,17.404623,8.484754,4.650298,2.845248,1.893008,1.329408,0.963175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-05-08 06:04:43,2.284357,0.571089,0.142772,0.035693,0.008923,0.002231,0.002231,2.284357,2.284357,0.000002,4.379452,-26.515360,0.005167,0.250000,0.250000,0.250000
2013-05-08 07:38:43,6.015946,45.743889,354.032727,2772.669899,21883.399622,173577.399114,173577.399114,6.015946,6.015946,0.002786,12.160964,52.394932,0.031003,7.603774,7.831677,7.892537
2013-05-08 12:52:42,26.194345,178.470120,1267.441102,9154.667055,66698.019196,488438.236707,488438.236707,26.194345,26.194345,0.001659,15.518885,56.888097,0.067172,6.813307,7.222953,7.285685
2013-05-08 15:46:41,48.732943,19.797758,9.327790,4.592509,2.287331,1.142550,1.142550,48.732943,48.732943,0.000311,6.140364,0.578753,0.007751,0.406250,0.492347,0.498057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-05-13 05:38:28,2.610693,0.489505,0.101980,0.022946,0.005418,0.001315,0.001315,2.610693,2.610693,0.000003,4.379452,-28.812101,0.005167,0.187500,0.225000,0.236111
2013-05-13 05:43:28,3.892509,0.988569,0.356152,0.156204,0.074216,0.036335,0.036335,3.892509,3.892509,0.000012,9.150664,-14.396783,0.015501,0.253967,0.438588,0.475126
2013-05-13 05:45:28,1.731690,0.502006,0.169830,0.061383,0.022731,0.008488,0.008488,1.731690,1.731690,0.000002,4.379452,-20.711883,0.005167,0.289894,0.361441,0.370311
2013-05-13 05:47:28,2.610693,1.305347,0.652673,0.326337,0.163168,0.081584,0.081584,2.610693,2.610693,0.000021,7.389752,-10.883941,0.010334,0.500000,0.500000,0.500000


In [ ]:
plt.figure(figsize=(8, 5))
plt.rc('text', usetex=True)
plt.yscale('log')
plt.scatter(dsd_results['shp_246'], dsd_results['N0_246'], marker='+')
plt.ylim([10 * 3, 10 * 8])
plt.xlim([-3, 7])
plt.xlabel('$\mu $', size=14)
plt.ylabel('$ N_{0},\  \# \ \ m^{-3} mm^{-\mu -1} $', size=14)
plt.savefig('{0}/N0_shp_{1}_{2}.png'.format(savefigs, dsd_number, str(num_gotas)))
plt.show()

In [ ]:

i = 0
for chunk in chunks:
    i += 1
    print (i)
    dsd_data_final = chunk.copy()
    dsd_data_final['Date_str'] = chunk[0].astype(str)
    dsd_data_final['Date'] = pd.to_datetime(dsd_data_final['Date_str'], format='%Y-%m-%d %H:%M:%S')
    dsd_data_final = dsd_data_final.drop_duplicates('Date', keep='first')

    dsd_data_final.set_index('Date', drop=True, inplace=True)
    dsd_data_final.drop(['Date_str'], axis=1, inplace=True)
    dsd_data_final.drop(range(43), axis=1, inplace=True)
    dsd_data_final.sort_index(inplace=True)

    idx_header = pd.MultiIndex.from_product([sorted(list_size), sorted(list_speed)], names=['Size', 'Speed'])
    dsd_data_final.columns = idx_header

    min_date = dsd_data_final.index[0]
    max_date = dsd_data_final.index[-1]

    sr_sum_drisd = dsd_data_final.sum(axis=1)
    idx_events = sr_sum_drisd[sr_sum_drisd > num_gotas].index

    delta_diam = pd.Series(sorted(list_size.values()), index=sorted(list_size)).astype(float)
    diameters = pd.Series(sorted(list_size.keys()), index=sorted(list_size)).astype(float)

    for event_ext in idx_events:
        sr_nd = pd.Series(index=sorted(list_size))
        sr_nt = pd.Series(index=sorted(list_size))
        sr_w = pd.Series(index=sorted(list_size))
        sr_R = pd.Series(index=sorted(list_size))
        sr_Z = pd.Series(index=sorted(list_size))

        for size in list_size:
            df_size = dsd_data_final.xs(size, level=0, axis=1)
            sr_event = df_size.loc[event_ext]
            dsize = list_size[size]

            nd = calc_nd(sr_event, list_speed, dsize)
            sr_nd.loc[size] = nd

            nt = calc_nt(sr_event, list_speed, dsize)
            sr_nt.loc[size] = nt

            W = calc_W(sr_event, list_speed, dsize)
            sr_w.loc[size] = W

            R = calc_R(sr_event, list_speed, dsize)
            sr_R.loc[size] = R

            Z = calc_R(sr_event, list_speed, dsize)
            sr_Z.loc[size] = Z

        # sr_nd.to_pickle(savepks + '/sr_nd_{0}.pkl'.format(event_ext))

        try:
            m0 = pd.Series(index=sorted(list_size))
            m0 = sr_nd * delta_diam
            M0 = m0.sum()
            dsd_results.loc[event_ext, 'M0'] = M0

            m1 = pd.Series(index=sorted(list_size))
            m1 = sr_nd * delta_diam * diameters
            M1 = m1.sum()
            dsd_results.loc[event_ext, 'M1'] = M1

            m2 = pd.Series(index=sorted(list_size))
            m2 = sr_nd * delta_diam * diameters ** 2
            M2 = m2.sum()
            dsd_results.loc[event_ext, 'M2'] = M2

            m3 = pd.Series(index=sorted(list_size))
            m3 = sr_nd * delta_diam * diameters ** 3
            M3 = m3.sum()
            dsd_results.loc[event_ext, 'M3'] = M3

            m4 = pd.Series(index=sorted(list_size))
            m4 = sr_nd * delta_diam * diameters ** 4
            M4 = m4.sum()
            dsd_results.loc[event_ext, 'M4'] = M4

            m5 = pd.Series(index=sorted(list_size))
            m5 = sr_nd * delta_diam * diameters ** 5
            M5 = m5.sum()
            dsd_results.loc[event_ext, 'M5'] = M5

            m6 = pd.Series(index=sorted(list_size))
            m6 = sr_nd * delta_diam * diameters ** 6
            M6 = m6.sum()
            dsd_results.loc[event_ext, 'M6'] = M6

            ## MICROFISICA DE LA PRECIPITACION POR DISDROMETRO ###

            NT_dsd = sr_nt.sum()
            NT_dsd_1 = M0
            W_dsd = (math.pi / 6.0) * 10 ** (-3) * sr_w.sum()
            R_dsd = 6*math.pi * 10 ** (-4.0) * sr_R.sum()
            Z_dsd = 10*np.log10(sr_Z.sum())
            Z_dsd_cal = 10*np.log10(M6)
            Da_dsd = M1 / M0
            De_dsd = M3 / M2
            Dm_dsd = M4 / M3

            dsd_results.loc[event_ext, 'NT_dsd'] = NT_dsd
            dsd_results.loc[event_ext, 'NT_dsd_1'] = NT_dsd_1
            dsd_results.loc[event_ext, 'W_dsd'] = W_dsd
            dsd_results.loc[event_ext, 'Z_dsd'] = Z_dsd
            dsd_results.loc[event_ext, 'Z_dsd_cal'] = Z_dsd_cal
            dsd_results.loc[event_ext, 'R_dsd'] = R_dsd
            dsd_results.loc[event_ext, 'Da_dsd'] = Da_dsd
            dsd_results.loc[event_ext, 'De_dsd'] = De_dsd
            dsd_results.loc[event_ext, 'Dm_dsd'] = Dm_dsd

            ###    MODELO MARSHALL AND PALMER 1948  ###

            mmt_nb = 1.0
            slp_mp = ((8000.0 * special.gamma(mmt_nb + 1.0)) / (M1)) ** (1.0 / (mmt_nb + 1.0))
            M_P = pd.Series(index=sorted(list_size))
            M_P = 8000.0 * np.exp(-slp_mp * diameters)


            NT_mp = 8000.0 / slp_mp
            W_mp = (math.pi / 6.0) * 10 * (-3.) * (8000.0 * (special.gamma(3. + 1.) / (slp_mp * (3. + 1.))))
            Z_mp = 8000.0 * (special.gamma(6. + 1.) / (slp_mp ** (6. + 1.)))

            R_mp = 6*10*(-4)*math.pi*8000.*3.778(special.gamma(3.+0.67+1.)/(slp_mp**(3.+0.67+1.)))

            #### VELOCIDAD TERMINAL = 9.65 - 10.3*e^(-0.6D)

            K1 = 6 * 10 ** (-4) * math.pi * 8000.0
            R_mp_2 = K1 * (9.65 * (special.gamma(3. + 1) / (slp_mp ** (3. + 1.))) -
                           10.3 * (special.gamma(3. + 1) / ((slp_mp + 0.6) ** (3. + 1.))))

            #### VELOCIDAD TERMINAL = POLINOMICA

            R_mp_3 = K1 * (-0.1021*(special.gamma(3. + 1.) / (slp_mp ** (3. + 1.))) +
                           4.932*(special.gamma(3. + 2.) / (slp_mp ** (3. + 2.)))
                           - 0.9551*(special.gamma(3. + 3.) / (slp_mp ** (3. + 3.)))
                           + 0.07934*(special.gamma(3. + 4.) / (slp_mp ** (3. + 4.)))
                           - 0.002362*(special.gamma(3. + 5.) / (slp_mp ** (3. + 5.))))
            Da_mp = 8000.0 * (special.gamma(1. + 1.) / (slp_mp ** (1. + 1.))) / (8000.0 / slp_mp)
            De_mp = 8000.0 * (special.gamma(3. + 1.) / (slp_mp ** (3. + 1.))) / (
                8000.0 * (special.gamma(2. + 1.) / (slp_mp ** (2. + 1.))))
            Dm_mp = 8000.0 * (special.gamma(4. + 1.) / (slp_mp ** (4. + 1.))) / (
                8000.0 * (special.gamma(3. + 1.) / (slp_mp ** (3. + 1.))))

            dsd_results.loc[event_ext, 'slp_mp'] = slp_mp
            dsd_results.loc[event_ext, 'NT_mp'] = NT_mp
            dsd_results.loc[event_ext, 'W_mp'] = W_mp
            dsd_results.loc[event_ext, 'Z_mp'] = 10*np.log10(Z_mp)
            dsd_results.loc[event_ext, 'R_mp'] = R_mp
            dsd_results.loc[event_ext, 'R_mp_2'] = R_mp_2
            dsd_results.loc[event_ext, 'R_mp_3'] = R_mp_3
            dsd_results.loc[event_ext, 'Da_mp'] = Da_mp
            dsd_results.loc[event_ext, 'Da_mp'] = De_mp
            dsd_results.loc[event_ext, 'Da_mp'] = Dm_mp

            #     Modelo Exponencial

            m = 2.0
            n = 1.0
            slopexp = ((M1 * special.gamma(m + 1.0)) / (M2 * special.gamma(n + 1.0))) ** (1 / (m - n))
            N_0exp = M1 * (slopexp ** (n + 1.0)) / (special.gamma(n + 1.0))
            Exp = pd.Series(index=sorted(list_size))
            Exp = N_0exp * np.exp(-slopexp * diameters)

            Nt_exp = N_0exp / slopexp
            W_exp = (math.pi / 6.0) * 10 * (-3.) * N_0exp * special.gamma(3. + 1.) / (slopexp * (3. + 1.))
            Z_exp = N_0exp * special.gamma(6. + 1.) / (slopexp ** (6. + 1.))

            ## VELOCIDAD TERMINAL = aD^b
            R_exp = 6*10*(-4)*math.pi*N_0exp*3.778(special.gamma(3.+0.67+1)/(slopexp**(3.+0.67+1)))

            #### VELOCIDAD TERMINAL = 9.65 - 10.3*e^(-0.6D)

            K = 6 * 10 ** (-4) * math.pi * N_0exp
            R_exp_2 = K * (9.65 * (special.gamma(3. + 1) / (slopexp ** (3. + 1.))) -
                           10.3 * (special.gamma(3. + 1) / ((slopexp + 0.6) ** (3. + 1.))))

            #### VELOCIDAD TERMINAL = POLINOMICA

            R_exp_3 = K * (-0.1021*(special.gamma(3. + 1.) / (slopexp ** (3. + 1.))) +
                           4.932*(special.gamma(3. + 2.) / (slopexp ** (3. + 2.)))
                           - 0.9551*(special.gamma(3. + 3.) / (slopexp ** (3. + 3.)))
                           + 0.07934*(special.gamma(3. + 4.) / (slopexp ** (3. + 4.)))
                           - 0.002362*(special.gamma(3. + 5.) / (slopexp ** (3. + 5.))))

            Da_exp = N_0exp * (special.gamma(1. + 1.) / (slopexp ** (1. + 1.))) / (N_0exp / slopexp)
            De_exp = N_0exp * (special.gamma(3. + 1.) / (slopexp ** (3. + 1.))) / (
                N_0exp * (special.gamma(2. + 1.) / (slopexp ** (2. + 1.))))
            Dm_exp = N_0exp * (special.gamma(4. + 1.) / (slopexp ** (4. + 1.))) / (
                N_0exp * (special.gamma(3. + 1.) / (slopexp ** (3. + 1.))))

            dsd_results.loc[event_ext, 'slp_exp'] = slopexp
            dsd_results.loc[event_ext, 'N0_exp'] = N_0exp
            dsd_results.loc[event_ext, 'NT_exp'] = Nt_exp
            dsd_results.loc[event_ext, 'W_exp'] = W_exp
            dsd_results.loc[event_ext, 'Z_exp'] = 10*np.log10(Z_exp)
            dsd_results.loc[event_ext, 'R_exp'] = R_exp
            dsd_results.loc[event_ext, 'R_exp_2'] = R_exp_2
            dsd_results.loc[event_ext, 'R_exp_3'] = R_exp_3
            dsd_results.loc[event_ext, 'Da_exp'] = Da_exp
            dsd_results.loc[event_ext, 'De_exp'] = De_exp
            dsd_results.loc[event_ext, 'Dm_exp'] = Dm_exp

            #      Modelo Gamma 246

            eta246 = M4 ** 2. / (M2 * M6)
            shape246 = ((7.0 - 11.0 * eta246) - (eta246 * 2 + 14 * eta246 + 1.0) * 0.5) / (2.0 * (eta246 - 1.0))
            slope246 = ((M2 / M4) * (shape246 + 3.0) * (shape246 + 4.0)) ** 0.5
            N0_246 = (M2 * slope246 ** (shape246 + 3.0)) / (special.gamma(shape246 + 3.0))
            gamma246 = pd.Series(index=sorted(list_size))
            gamma246 = N0_246 * (diameters ** shape246) * np.exp(-slope246 * diameters)
            NT_246 = N0_246 * (special.gamma(shape246 + 1) / (slope246 ** (shape246 + 1)))
            W_246 = (math.pi / 6.0) * 10 * (-3.) * N0_246 * (special.gamma(shape246 + 4) / (slope246 * (shape246 + 4)))
            Z_246 = N0_246 * (special.gamma(shape246 + 7) / (slope246 ** (shape246 + 7)))

            ## VELOCIDAD TERMINAL = aD^b

            R_246 = 6*10*(-4)*math.pi*N0_246*3.778(special.gamma(3.+shape246+0.67+1)/(slope246**(3.+shape246+0.67+1)))

            ## VELOCIDAD TERMINAL = 9.65 - 10.3*e^(-0.6*D)

            K = 6 * 10 ** (-4) * math.pi * N0_246
            R_246_2 = K * (9.65 * (special.gamma(3. + shape246 + 1) / (slope246 ** (3. + shape246 + 1))) -
                    10.3 * (special.gamma(3. + shape246 + 1) / ((slope246 + 0.6) ** (3. + shape246 + 1))))

            ## VELOCIDAD TERMINAL = POLINOMICA

            R_246_3 = K * (-0.1021*(special.gamma(3. + shape246 + 1.) / (slope246 ** (3. + shape246 + 1.))) +
                           4.932*(special.gamma(3. + shape246 + 2.) / (slope246 ** (3. + shape246 + 2.)))
                           - 0.9551*(special.gamma(3. + shape246 + 3.) / (slope246 ** (3. + shape246 + 3.)))
                           + 0.0793*(special.gamma(3. + shape246 + 4.) / (slope246 ** (3. + shape246 + 4.)))
                           - 0.002362*(special.gamma(3. + shape246 + 5.) / (slope246 ** (3. + shape246 + 5.))))

            Da_246 = (N0_246 * (special.gamma(shape246 + 2.) / (slope246 ** (shape246 + 2.)))) / (
                N0_246 * (special.gamma(shape246 + 1.) / (slope246 ** (shape246 + 1.))))
            De_246 = (N0_246 * (special.gamma(shape246 + 4.) / (slope246 ** (shape246 + 4.)))) / (
                N0_246 * (special.gamma(shape246 + 3.) / (slope246 ** (shape246 + 3.))))
            Dm_246 = (N0_246 * (special.gamma(shape246 + 5.) / (slope246 ** (shape246 + 5.)))) / (
                N0_246 * (special.gamma(shape246 + 4.) / (slope246 ** (shape246 + 4.))))

            dsd_results.loc[event_ext, 'slp_246'] = slope246
            dsd_results.loc[event_ext, 'shp_246'] = shape246
            dsd_results.loc[event_ext, 'N0_246'] = N0_246
            dsd_results.loc[event_ext, 'NT_246'] = NT_246
            dsd_results.loc[event_ext, 'W_246'] = W_246
            dsd_results.loc[event_ext, 'Z_246'] = 10*np.log10(Z_246)
            dsd_results.loc[event_ext, 'R_246'] = R_246
            dsd_results.loc[event_ext, 'R_246_2'] = R_246_2
            dsd_results.loc[event_ext, 'R_246_3'] = R_246_3
            dsd_results.loc[event_ext, 'Da_246'] = Da_246
            dsd_results.loc[event_ext, 'De_246'] = De_246
            dsd_results.loc[event_ext, 'Dm_246'] = Dm_246

            dsd_results.append(dsd_results)

            # plt.figure(figsize=(12, 9))
            # plt.rc('text', fontsize=18, usetex=True)
            # ax = sr_nd[sr_nd > 1.].plot(style='x-', logy=True, grid=True, ylim=[10 * 0, 10 * 6],
            #                             xlim=[0, 5], legend=True, label='$ Observado $', fontsize=16)
            # M_P[M_P > 1.].plot(style='--', logy=True, label='$ Marshall - Palmer $', legend=True, ylim=[10 * 0, 10 * 6],
            #                    xlim=[0, 8])
            # Exp[Exp > 1.].plot(style='-.', logy=True, label='$ Exponencial $', legend=True, ylim=[10 * 0, 10 * 6],
            #                    xlim=[0, 8])
            #
            # gamma246.plot(style='+', color='k', logy=True, label='$Gamma$', legend=True,
            #               ylim=[10 * 0, 10 * 6], xlim=[0, 8])
            #
            # ax.set_xlabel('$Diametro, mm$', fontsize=16)
            # ax.set_ylabel('$N(D), \ \# \ m^{-3} mm^{-1} $', fontsize=16)
            # plt.savefig(savefigs + '/fig_{0}'.format(event_ext))
            # plt.close('all')
        except (ZeroDivisionError,OverflowError) as e:
            print (e)
            continue

dsd_results.sort_index(inplace=True)
dsd_results.dropna(how='all', inplace=True)
xls_data = pd.ExcelWriter('{0}/{1}/dsd_results_{2}_{3}.xlsx'.format(path_results, dsd_number, dsd_number, num_gotas))
dsd_results.to_excel(xls_data)
xls_data.save()

## GRAFICO DE N0 Vs Shape

plt.figure(figsize=(8, 5))
plt.rc('text', usetex=True)
plt.yscale('log')
plt.scatter(dsd_results['shp_246'], dsd_results['N0_246'], marker='+')
plt.ylim([10 * 3, 10 * 8])
plt.xlim([-3, 7])
plt.xlabel('$\mu $', size=14)
plt.ylabel('$ N_{0},\  \# \ \ m^{-3} mm^{-\mu -1} $', size=14)
plt.savefig('{0}/N0_shp_{1}_{2}.png'.format(savefigs, dsd_number, str(num_gotas)))
plt.show()

### GRAFICO DE N0 Vs Slope

plt.figure(figsize=(8, 5))
plt.rc('text', usetex=True)
plt.yscale('log')
plt.scatter(dsd_results['slp_246'], dsd_results['N0_246'], marker='+')
plt.ylim([10 * 2, 10 * 7])
plt.xlim([0, 7])
plt.xlabel('$\Lambda, mm^{-1} $', size=14)
plt.ylabel('$ N_{0},\  \# \ \ m^{-3} mm^{-\mu -1} $', size=14)
plt.savefig('{0}/N0_slp_{1}_{2}.png'.format(savefigs, dsd_number, str(num_gotas)))
plt.show()

### GRAFICO DE shape Vs Slope

plt.figure(figsize=(8, 5))
plt.rc('text', usetex=True)
z = np.polyfit(dsd_results['shp_246'], dsd_results['slp_246'], 2)
z1 = np.polyfit(dsd_results['slp_246'], dsd_results['shp_246'], 2)
plt.scatter(dsd_results['slp_246'], dsd_results['shp_246'], marker='+')
plt.xlim([0, 6])
plt.ylim([-2.5, 4])
plt.xlabel('$\Lambda, mm^{-1} $', size=14)
plt.ylabel('$ \mu $', size=14)
plt.savefig('{0}/slp_shp_{1}_{2}.png'.format(savefigs, dsd_number, str(num_gotas)))
plt.show()

np.savetxt(
    '{0}/{1}/CG_relation_z_{2}_{3}.xlsx'.format(path_results, dsd_number, dsd_number, num_gotas), z, delimiter=",")
np.savetxt(
    '{0}/{1}/CG_relation_z1_{2}_{3}.xlsx'.format(path_results, dsd_number, dsd_number, num_gotas), z1, delimiter=",")

In [ ]:
spark.stop